## Step 1 Installing Required libraries
This command installs the necessary libraries:

 - einops: A flexible library for tensor operations, providing simple and readable syntax for reshaping, rearranging, and aggregating tensors.requests: For making HTTP requests to load content from websites.
 - langchain: A framework for building applications using large language models (LLMs), enabling easy chaining of tasks like prompting, retrieval, and generation.
 - pymilvus: A client for interacting with the Milvus vector database.
 - sentence-transformers: A library used for generating embeddings (vector representations of text).
 - langchain-hugginfface: A dedicated LangChain package for integrating Hugging Face models, including embedding and LLM models.
 

In [9]:
%%capture
!pip install -q einops langchain-community pymilvus sentence-transformers langchain-huggingface


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
import os
#from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

## Configure Milvus Connection

Here is the DB connection. Make sure to give a unique name to your collection

In [11]:
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "splunk_appdynamics"

## Initialize the connection

In [12]:

model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

In [15]:
query="What is an appdynamics agent ?"
results = store.similarity_search_with_score(query, k=5, return_metadata=True)
for result in results:
    print(result[0].metadata['source'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer
https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer
https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer
https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer
https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer/deploy-an-agent-using-the-agent-installer-api


In [16]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [17]:
docs = retriever.get_relevant_documents(query)
docs

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'source': 'https://docs.appdynamics.com/appd/24.x/latest/en/application-monitoring/install-app-server-agents/agent-installer', 'title': 'Agent Installer', 'language': 'en'}, page_content='Deploys Java and\xa0Machine Agents.\xa0Automatically instruments applications.Assigns\xa0unique\xa0names to tiers and nodes.Upgrades and rollbacks the agents.\n\nDeploys Machine Agents.\xa0\n\nYou can deploy other agents using the Getting Started Wizard.\xa0Agent Installer OverviewThe Agent Installer works with these items:\n\n\n\nAgent Installer\xa0-\xa0Executable installer inside the\xa0appdynamics-zero-agent-<version>.zip.\xa0file (representing the Agent Installer Platform). The Agent Installer installs the Agent Installer Platform\xa0with Java, and Machine Agents on a system.\xa0You access the installer from the Controller UI.Decorator - Library responsible for automatically instrumenting new processes with Splunk AppDynamics APM agents. The library auto-instruments the applica